In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 280 ms (started: 2021-09-05 15:30:03 +08:00)


In [ ]:
%%bash

# 增加更新
git add *.ipynb */*.ipynb

git remote -v

git commit -m '更新 #3 Sept 05, 2021'

#git push origin master
git push

在构建神经网络时将梯度归零是有益的。这是因为默认情况下，每当调用 .backward() 时，梯度都会在缓冲区中累积（即不会被覆盖）。

# 介绍

在训练神经网络时，模型能够通过梯度下降来提高其准确性。简而言之，梯度下降是通过调整模型中的权重和偏差来最小化损失（或错误）的过程。

torch.Tensor 是 PyTorch 的核心类。创建张量时，如果将其属性 .requires_grad 设置为 True，则该包会跟踪对其进行的所有操作。这发生在随后的向后传递中。此张量的梯度将累积到 .grad 属性中。当在损失张量上调用 .backward() 时，计算所有梯度的累积（或总和）。

在某些情况下，可能需要将张量的梯度归零。例如：当您开始训练循环时，您应该将梯度归零，以便您可以正确执行此跟踪。在这个秘籍中，我们将学习如何使用 PyTorch 库将梯度归零。我们将通过在 PyTorch 内置的 CIFAR10 数据集上训练神经网络来演示如何做到这一点。

# 步骤

步骤 1 到 4 设置我们的数据和神经网络以进行训练。 将梯度归零的过程发生在第 5 步。如果您已经构建了数据和神经网络，请跳至第 5 步。
* 导入所有必要的库以加载我们的数据
* 加载和规范化数据集
* 构建神经网络
* 定义损失函数
* 训练网络时将梯度归零

1. 导入必要的库来加载我们的数据

对于这个秘籍，我们将只使用 torch 和 torchvision 来访问数据集。

In [2]:
import torch

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

time: 744 ms (started: 2021-09-05 15:30:07 +08:00)


2. 加载和规范化数据集

PyTorch 具有各种内置数据集（有关更多信息，请参阅加载数据配方）。

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Using downloaded and verified file: ./data/cifar-10-python.tar.gz
Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
time: 3.11 s (started: 2021-09-05 18:54:36 +08:00)


3. 构建神经网络

我们将使用卷积神经网络。 要了解更多信息，请参阅定义神经网络食谱。

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

time: 1.63 ms (started: 2021-09-05 18:55:16 +08:00)


4. 定义一个损失函数和优化器

让我们使用带有动量的分类交叉熵损失和 SGD。

In [7]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

time: 28.1 ms (started: 2021-09-05 18:55:39 +08:00)


5. 在训练网络时将梯度归零

这是事情开始变得有趣的时候。 我们只需要遍历我们的数据迭代器，并将输入提供给网络并进行优化。

请注意，对于每个数据实体，我们将梯度归零。 这是为了确保我们在训练神经网络时不会跟踪任何不必要的信息。

In [8]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

/data/miniconda3/envs/torch19/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,  2000] loss: 2.195
[1,  4000] loss: 1.853
[1,  6000] loss: 1.677
[1,  8000] loss: 1.586
[1, 10000] loss: 1.503
[1, 12000] loss: 1.462
[2,  2000] loss: 1.359
[2,  4000] loss: 1.355
[2,  6000] loss: 1.330
[2,  8000] loss: 1.317
[2, 10000] loss: 1.282
[2, 12000] loss: 1.265
Finished Training
time: 1min 56s (started: 2021-09-05 18:57:35 +08:00)


您也可以使用 model.zero_grad()。 只要您的所有模型参数都在该优化器中，这与使用 optimizer.zero_grad() 相同。 使用您的最佳判断来决定使用哪一个。

恭喜！ 您已成功将 PyTorch 的梯度归零。